データとlibをダウンロード

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [4]:
ls

adc.json  lib.zip  sample_data/


In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1eDylstW-XZ7tfPmUrG__hEsWFxTWEMZE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('stanford-dogs-dataset.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "stanford-dogs-dataset.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [10]:
%cd data/images/Images

/content/data/images/Images


In [0]:
os.mkdir("./train/")
os.mkdir("./val/")

In [0]:
import glob
import shutil

foruda = os.listdir(path='.')
path = os.getcwd()  

for a in foruda:
 if a.find("-") != -1:
  files = os.listdir(path + "/" + a)  
  count = len(files)  
  files = glob.glob("./"+a +"/*")
  i = 0
  for file in files:
    i = 1 + i
    if i < count*1/4:
     data_dir = "./val/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './val/'+ a + "/")
    else:
     data_dir = "./train/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './train/'+ a + "/")
  shutil.rmtree("./"+a)

In [15]:
cd train

/content/data/images/Images/train


In [16]:
cd ../val

/content/data/images/Images/val


In [17]:
 cd /content

/content


*モデルの適用*

In [18]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.376s
     |████████████████████████████████| 686kB 21.5MB/s 
     |████████████████████████████████| 204kB 54.0MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=acea6c04c5b32eccfab463e365b1891cd15d4c1e048ef40a4d0f1f2abe950ee5
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=0d6dd0100088f15fa382a9535f1be439f8046bd9428fde539d8e7a633efe39db
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 108 ms, sys: 76.7 ms, total: 185 ms
Wall time: 21.2 s


In [19]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 14.7 ms, sys: 17.4 ms, total: 32.1 ms
Wall time: 7.09 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [21]:
%%time
net = network.ResNet50(class_size=120)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/train', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/val', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 55.3 ms, sys: 11 ms, total: 66.3 ms
Wall time: 67.4 ms


In [23]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.42 ms, sys: 31.3 ms, total: 33.8 ms
Wall time: 57.5 ms


In [24]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-23 03:50:27.488633
val Loss: 15.3875 Acc: 0.0075
----------
Epoch 2/50 2020-01-23 03:50:54.906060
train Loss: 5.3514 Acc: 0.0001
val Loss: 4.8231 Acc: 0.0116
----------
Epoch 3/50 2020-01-23 03:53:55.813135
train Loss: 4.7612 Acc: 0.0000
val Loss: 4.9448 Acc: 0.0116
----------
Epoch 4/50 2020-01-23 03:56:59.169236
train Loss: 4.8055 Acc: 0.0115
val Loss: 4.8309 Acc: 0.0116
----------
Epoch 5/50 2020-01-23 04:00:02.227797
train Loss: 4.8080 Acc: 0.0115
val Loss: 4.7954 Acc: 0.0114
----------
Epoch 6/50 2020-01-23 04:03:05.796060
train Loss: 4.8054 Acc: 0.0113
val Loss: 4.8077 Acc: 0.0116
----------
Epoch 7/50 2020-01-23 04:06:09.056202
train Loss: 4.9578 Acc: 0.0113
val Loss: 7.1062 Acc: 0.0097
----------
Epoch 8/50 2020-01-23 04:09:12.590074
train Loss: 4.8242 Acc: 0.0000
val Loss: 45.1711 Acc: 0.0099
----------
Epoch 9/50 2020-01-23 04:12:15.625956
train Loss: 4.7963 Acc: 0.0106
val Loss: 4.8105 Acc: 0.0105
----------
Epoch 10/50 2020-01-23 04:15:19.00381

In [25]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 0 %
CPU times: user 5.61 s, sys: 4.09 s, total: 9.7 s
Wall time: 27.7 s
